In [ ]:
# default_exp data.transforms

# Data Transforms

> API details.

In [ ]:
# TODO: Add type hints

In [ ]:
# export
import re

import pandas as pd

from random import shuffle
from tokenizers import Tokenizer
from typing import Callable, Optional

In [ ]:
#hide
from nbdev.showdoc import *

## Semantic Preserving

In [ ]:
# export
# TODO: Write test case
def java_comment_remover(mthd: str) -> str:
    """
    Remove all comments from a given java method. Code from https://stackoverflow.com/a/241506/5768407.

    :param mthd: the method to have its comments removed
    :returns: returns the method with its comments removed
    """
    def replacer(match):
        s = match.group(0)
        if s.startswith('/'):
            return " " # note: a space and not an empty string
        else:
            return s
    
    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )
    return re.sub(pattern, replacer, mthd)

## Non-Semantic Preserving

In [ ]:
# export
# TODO: add test cases
def randomize_lines(mthd: str) -> str:
    """
    Randomize the lines in a given method.

    :param mthd: the method to have its lines randomized
    :returns: returns the method with its lines randomized
    """
    mthd = mthd.split('<n>')
    shuffle(mthd)
    
    return '<n>'.join(mthd)

def randomize_tokens(mthd: str, tokenizer: Tokenizer) -> str:
    """
    Randomize the tokens in a given method.

    :param mthd: the method to have its tokens randomized
    :param tokenizer: the tokenizer that will tokenize the given method
    :returns: returns the method with its tokens randomized
    """
    mthd = tokenizer.encode(mthd).ids
    shuffle(mthd)
    
    return tokenizer.decode(mthd)

In [ ]:
# hide
from icodegen.data import *

df_fake = pd.DataFrame([
    '''public void setPipelines(java.util.Collection<Pipeline> pipelines) {
        if (pipelines == null) {
            this.pipelines = null;
            return;
        }

        this.pipelines = new com.amazonaws.internal.SdkInternalList<Pipeline>(pipelines);
    }
    '''
], columns = ['code']); df_fake
df_fake = beautify_code(df_fake)
df_fake = replace_special_tokens(df_fake, java_special_tokens)
tokenizer = train_tokenizer(df_fake)

In [ ]:
# export
# TODO: add docs and test case
def transform_df(df: pd.DataFrame, transform: Callable, n: Optional[int] = None) -> pd.DataFrame:
    """
    Transform the given pandas dataframe using the given transformation.

    :param df: the dataframe containing each method to be transformed
    :param transform: the transformation that will be applied to each method in the dataframe
    :param n: the number of methods to evaluate. If none, the entire dataframe will be used
    :returns: returns a modified dataframe with the methods transformed
    """
    if n is None: n = len(df)

    df = df.iloc[:n].copy()
    df.code = df.code.apply(transform)

    return df